In [1]:
from pathlib import Path

DIR = Path.home() / "Desktop" / "xdsl_jit"

 - hash name
 - get python types from function type
 - add a compile function that takes (name, input types, output types) and returns a callable

In [2]:
from xdsl.builder import ImplicitBuilder

from xdsl.dialects import func, arith, memref
from xdsl.dialects.builtin import ModuleOp, i64, f64


def get_module() -> ModuleOp:
    memref_t = memref.MemRefType.from_element_type_and_shape(f64, [100])
    module = ModuleOp([])
    with ImplicitBuilder(module.body):
        f = func.FuncOp("hello", ((i64, i64), (i64,)))
        with ImplicitBuilder(f.body) as (lhs, rhs):
            res = arith.Addi(lhs, rhs).result
            func.Return(res)

        g = func.FuncOp("dist", ((memref_t, memref_t), (i64,)))
        with ImplicitBuilder(g.body) as (lhs, rhs):
            res = arith.Constant.from_int_and_width(1, 64)
            func.Return(res)

    return module

str(get_module())

'builtin.module {\n  func.func @hello(%0 : i64, %1 : i64) -> i64 {\n    %2 = arith.addi %0, %1 : i64\n    func.return %2 : i64\n  }\n  func.func @dist(%3 : memref<100xf64>, %4 : memref<100xf64>) -> i64 {\n    %5 = arith.constant 1 : i64\n    func.return %5 : i64\n  }\n}'

In [5]:
from xdsl.jit import jit_module

hey = jit_module(get_module(), "hello", c_types=((int, int), int))
dist_mlir = jit_module(get_module(), "dist", c_types=((int, int), int))
hey(5,6)

11

In [6]:
dist_mlir(1, 2)

1

In [ ]:
%time hey(5, 6)

In [ ]:
%time 1 + 2

In [ ]:
import numpy as np

In [ ]:
from typing import Any


def dist_np(lhs: np.ndarray[Any, np.dtype[np.float64]], rhs: np.ndarray[Any, np.dtype[np.float64]]) -> np.float64:
    diff = lhs - rhs
    res = np.linalg.norm(diff)
    return res


In [ ]:
n = 100
a = np.random.random(n)
b = np.random.random(n)

In [ ]:
dist_np(a, b)